# Detection and Measurements from sources Light Curve

- author Sylvie Dagoret-Campagne
- creation date 2024-06-01
- last update 2024-06-05
- affiliation : IJCLab
- Kernel **w_2024_16**

- Tutorial : https://github.com/rubin-dp0/tutorial-notebooks/blob/main/DP02_05_Introduction_to_Source_Detection.ipynb

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
props = dict(boxstyle='round', facecolor=None, alpha=0.1)
#props = dict(boxstyle='round')


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
import gc

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils
import lsst.daf.base as dafBase

In [ ]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.afw.display.rgb as afwRgb
import lsst.afw.image as afwImage
import lsst.geom as geom

In [ ]:
# Pipeline tasks
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.detection import SourceDetectionTask
from lsst.meas.deblender import SourceDeblendTask
from lsst.meas.base import SingleFrameMeasurementTask

In [ ]:

from astropy.wcs import WCS
from astropy.visualization import make_lupton_rgb
import gc

import lsst.afw.display as afwDisplay
from lsst.afw.image import MultibandExposure

In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

In [ ]:
# INSERT YOUR collection and tract
# for rehearsal use collection 2 which have CCDvisit
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240414/w_2024_15/PREOPS-5069' # COMPLETED

collectionn = collection
#collections = [collection1,collection2,collection3]
collections = [collection]
collectionStr = collectionn.replace("/", "_")
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"
instrument = 'LATISS'
skymapName = "latiss_v1"
where_clause = "instrument = \'" + instrument+ "\'"
tract = 3864 # mostly for light-curves
patch_sel = 236
band = 'g'
#tract = 5615
# tract = 5634 # interesting to view calib parameters
suptitle = collectionStr + f" inst = {instrument} tract = {tract}"

In [ ]:
#dataId = {"skymap": "latiss_v1", "tract": 5615, "instrument": "LATISS"}
dataId = {"skymap": skymapName, "tract": tract, "instrument": instrument}
repo = '/sdf/group/rubin/repo/oga/'
butler = dafButler.Butler(repo)
#t = Butler.get(table_sel, dataId=dataId, collections=collections)
registry = butler.registry

In [ ]:
skymap_auxtel ='latiss_v1'
skymap = butler.get('skyMap', skymap=skymap_auxtel, collections=collections)

In [ ]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

In [ ]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

In [ ]:
def cutout_coadd(butler, ra, dec, band='r', datasetType='deepCoadd',
                 skymap=None, cutoutSideLength=51, **kwargs):
    """
    Produce a cutout from a coadd at the given ra, dec position.

    Adapted from DC2 tutorial notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Helper object providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    band: string
        Filter of the image to load
    datasetType: string ['deepCoadd']
        Which type of coadd to load.  Doesn't support 'calexp'
    skymap: lsst.afw.skyMap.SkyMap [optional]
        Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)

    if skymap is None:
        skymap = butler.get("skyMap")

    # Look up the tract, patch for the RA, Dec
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    patch = tractInfo.getSequentialPatchIndex(patchInfo)

    coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
    parameters = {'bbox': bbox}

    cutout_image = butler.get(datasetType, parameters=parameters,
                              dataId=coaddId)

    return cutout_image

In [ ]:
def cutout_calexp(butler, ra, dec, visit, detector, cutoutSideLength=51, **kwargs):
    
    """
    Produce a cutout from a calexp at the given ra, dec position.

    Adapted from cutout_coadd which was adapted from a DC2 tutorial
    notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Helper object providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    visit: int
        Visit id of the calexp's visit
    detector: int
        Detector for the calexp
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    
    dataId = {'visit': visit, 'detector': detector}    
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)    
    calexp_wcs = butler.get('calexp.wcs', **dataId)
    xy = geom.PointI(calexp_wcs.skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    parameters = {'bbox': bbox}
    cutout_image = butler.get('calexp', parameters=parameters, **dataId)

    return cutout_image

In [ ]:
def create_rgb(image, bgr="gri", stretch=1, Q=10, scale=None):
    """
    Create an RGB color composite image.

    Parameters
    ----------
    image : `MultibandExposure`
        `MultibandExposure` to display.
    bgr : sequence
        A 3-element sequence of filter names (i.e., keys of the exps dict)
        indicating what band to use for each channel. If `image` only has
        three filters then this parameter is ignored and the filters
        in the image are used.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    scale: list of 3 floats, each less than 1. (default: None)
        Re-scales the RGB channels.

    Returns
    -------
    rgb: ndarray
        RGB (integer, 8-bits per channel) colour image as an NxNx3 numpy array.
    """

    # If the image only has 3 bands, reverse the order of the bands
    #   to produce the RGB image
    if len(image) == 3:
        bgr = image.filters

    # Extract the primary image component of each Exposure with the
    #   .image property, and use .array to get a NumPy array view.

    if scale is None:
        r_im = image[bgr[2]].array  # numpy array for the r channel
        g_im = image[bgr[1]].array  # numpy array for the g channel
        b_im = image[bgr[0]].array  # numpy array for the b channel
    else:
        # manually re-scaling the images here
        r_im = image[bgr[2]].array * scale[0]
        g_im = image[bgr[1]].array * scale[1]
        b_im = image[bgr[0]].array * scale[2]

    rgb = make_lupton_rgb(image_r=r_im,
                          image_g=g_im,
                          image_b=b_im,
                          stretch=stretch, Q=Q)
    # "stretch" and "Q" are parameters to stretch and scale the pixel values

    return rgb

## Get Pixel Scale

In [ ]:
import lsst.geom as geom
import lsst.sphgeom

skymap = butler.get('skyMap', skymap=skymapName, collections=collections )
tractInfo = skymap.generateTract(tract)
for patch in tractInfo:    
    patchID = patch.getSequentialIndex()
        
    ibb=patch.getInnerBBox()
    tWCS=tractInfo.getWcs()
       
    # loop on the 4 corners
    for icorn,corner in enumerate(ibb.getCorners()):
        p = geom.Point2D(corner.getX(), corner.getY())
        coord = tWCS.pixelToSky(p)

In [ ]:
tWCS

In [ ]:
#arcsec/pixel
pixel_scale = tWCS.getPixelScale().asArcseconds()

## Selected visits

In [ ]:
inputfilename = "sources_objectTable-t3864-bg-o547-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"
#inputfilename = "sources_objectTable-t3864-bg-o912-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"
df_myselectedvisits = pd.read_csv(inputfilename ,index_col=0)
tract = 3864
band = "g"
objectname = 547
path = f"calexp_t{tract}_b{band}_o{objectname}"
title = f"Auxtel Light Curves : tract = {tract}, band = {band}, object = {objectname} "
suptitle = inputfilename 

In [ ]:
if not os.path.exists(path):
    os.mkdir(path)

In [ ]:
df_myselectedvisits.sort_values("visit",inplace=True)
#df_myselectedvisits.sort_index(inplace=True)

In [ ]:
df_myselectedvisits

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(14,4))
df_myselectedvisits.plot.scatter(x="expMidptMjd",y="psfMag",ax=ax,s=20,c="zeroPoint",cmap="jet",grid=True,rot=45)
ax.set_title(title)
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(14,4))
df_myselectedvisits.plot.scatter(x="expMidptMjd",y="psfMagDiffMmag",ax=ax,s=20,c="zeroPoint",cmap="jet",grid=True,rot=45)
ax.set_title(title)
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

In [ ]:
bundle_selected_object = df_myselectedvisits[["visit","psfMagDiffMmag"]]
fig, ax = plt.subplots(1,1,figsize=(16,2))
bundle_selected_object.plot.bar(x="visit",ax=ax,color="b" ,rot=90,grid=True)
title = f"psfMagDiffMmag for object {objectname} for tract {tract} in band {band} (AUXTEL)"
ax.set_title(title)

In [ ]:
print(path)

In [ ]:
listOfVisitId = df_myselectedvisits["visit"].values
listOfVisitId

In [ ]:
# Create a basic schema to use with these tasks
schema = afwTable.SourceTable.makeMinimalSchema()
print(schema)

# Create a container which will be used to record metadata
#  about algorithm execution
algMetadata = dafBase.PropertyList()
print('algMetadata: ')
algMetadata

In [ ]:
# process the image
# Characterize the image properties
config = CharacterizeImageTask.ConfigClass()
config.psfIterations = 1
charImageTask = CharacterizeImageTask(config=config)

# Detect sources
config = SourceDetectionTask.ConfigClass()
# detection threshold in units of thresholdType
config.thresholdValue = 10
# units for thresholdValue
config.thresholdType = "stdev"
sourceDetectionTask = SourceDetectionTask(schema=schema, config=config)

# Deblend sources
sourceDeblendTask = SourceDeblendTask(schema=schema)

# Measure source properties
config = SingleFrameMeasurementTask.ConfigClass()
sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema,
                                                   config=config,
                                                   algMetadata=algMetadata)

In [ ]:
datasetType = 'calexp'
boxSize = 25
imMin, imMax, Q = -0.001, 0.004, 8
expMin, expMax = -25, 100


all_xytarget = []
all_cutout = []
all_psfSigma = []
all_psfMagDiffMmag = []
all_apFlux_50_0_instFlux = []                 
all_apFlux_50_0_instFluxErr = []  

all_fpspans = []
all_fp = []
all_hfpspans = []
all_hfp = []
all_hfpspans_mask = []

all_sourcemeas = []


for index,visitId in enumerate(listOfVisitId):

    print(f"==================={index}) ====== {visitId} =============================")
    dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}

    #retrieve the calexp
    calexp = butler.get('calexp', **dataId,collections=collections)

    # retrieve some information
    info_psf = calexp.getPsf()
    info_wcs = calexp.getWcs()
    info_photocalib = calexp.getPhotoCalib()
    
    # remove the detection plane
    calexp.mask.removeAndClearMaskPlane('DETECTED')

    # get the background
    bkgd = butler.get('calexpBackground', **dataId,collections=collections)
    
    # add the background
    calexp.maskedImage += bkgd.getImage()

    # Select the subimage
    row_source = df_myselectedvisits.iloc[index]
    x_target = row_source['x']
    y_target = row_source['y']
    ra_target= row_source['ra']
    dec_target= row_source['dec']
    psfSigma = row_source['psfSigma']
    psfMagDiffMmag = row_source['psfMagDiffMmag'] 
    apFlux_50_0_instFlux = row_source['apFlux_50_0_instFlux']
    apFlux_50_0_instFluxErr = row_source['apFlux_50_0_instFluxErr']

    all_xytarget.append((x_target,y_target))

    print(f">>> target ({x_target},{y_target}),......, psfSigma = {psfSigma}")
    xSrc = x_target
    ySrc = y_target
    targetPoint = geom.Point2D(xSrc,ySrc)
    
    minBbox = geom.Point2I(int(xSrc) - boxSize ,int(ySrc) - boxSize)
    maxBbox = geom.Point2I(int(xSrc) + boxSize, int(ySrc) + boxSize)
    srcBbox = geom.Box2I(minBbox, maxBbox)
    # Make the cutout

    # two ways to make cutout ExposureF or Factory
    subimg = afwImage.ExposureF(calexp, srcBbox, afwImage.PARENT, True)
    
    # Generate the cutout image
    cutout = calexp.Factory(calexp, srcBbox, origin=afwImage.LOCAL, deep=False)
    #extent = (xmin,ymin,xmin+width,ymin+height)
    all_cutout.append(cutout)
    all_psfSigma.append(psfSigma)
    all_psfMagDiffMmag.append(psfMagDiffMmag)
    all_apFlux_50_0_instFlux.append(apFlux_50_0_instFlux)                
    all_apFlux_50_0_instFluxErr.append(apFlux_50_0_instFluxErr)  

        
    
    print("*****************************")
    print("* 1) Image Characterization *")  
    print("*****************************")
    # 1) Image characterization (this cell may take a few seconds)
    #result = charImageTask.run(cutout)
    result = charImageTask.run(calexp)
    point = geom.Point2D(x_target, y_target)
    # Get the PSF at our point of interest
    psf = calexp.getPsf()
    sigma = psf.computeShape(point).getDeterminantRadius()
    pixelScale = calexp.getWcs().getPixelScale().asArcseconds()

    # The factor of 2.355 converts from std to fwhm
    print('>>>>>>>>>>>>>>>   psf fwhm = {:.2f} arcsec'.format(sigma*pixelScale*2.355))

    print("*****************************")
    print("* 2) Sources Detection      *")  
    print("*****************************")
    # 2) Source detection (this cell may take a few seconds)
    tab = afwTable.SourceTable.make(schema)
    result = sourceDetectionTask.run(tab, calexp)
    
    sources = result.sources
    fpset = result.positive

    # work around footprint
    NFP = len(fpset.getFootprints())
    distance_to_target = np.zeros(NFP)
    for idx,fp in enumerate(fpset.getFootprints()):
        distance_to_target[idx] =  np.sqrt(fp.getCentroid().distanceSquared(targetPoint))
    idx_min_fp = np.where( distance_to_target == distance_to_target.min())[0][0]
    fpsel = fpset.getFootprints()[idx_min_fp]
    print(f" >>>>> footprint at index {idx_min_fp} :",fpsel)
    fpspans = fpsel.getSpans()
    all_fpspans.append(fpspans)
    all_fp.append(fpsel) 
    
    # convert fpset into Heavy Footprint
    fpset.makeHeavy(calexp.getMaskedImage())
    # This means we have to redefine fps:
    hfps = fpset.getFootprints()
    NHFP = len(hfps) 
    distance_to_target = np.zeros(NHFP)
    for idx,hfp in enumerate(hfps):
        distance_to_target[idx] =  np.sqrt(hfp.getCentroid().distanceSquared(targetPoint))
    idx_min_hfp = np.where( distance_to_target == distance_to_target.min())[0][0]
    hfpsel = hfps [idx_min_hfp]
    print(f" >>>>> heavy footprint at index {idx_min_hfp} :",hfpsel)

    all_hfpspans.append(hfpsel.getSpans().unflatten(hfpsel.getImageArray()))
    all_hfp.append(hfpsel)

    hfpmask = hfpsel.getSpans().unflatten(hfpsel.getMaskArray())
    all_hfpspans_mask.append(hfpmask)
    

    #Next we run the `SourceDeblendTask` and `SingleFrameMeasurementTask`. 
    # A deeper investigation of these tasks is beyond the scope of this notebook.
    
    print("*****************************")
    print("* 3) Sources Deblending     *")  
    print("*****************************")
    # 3) # Source deblending
    sourceDeblendTask.run(calexp, sources)

    print("*****************************")
    print("* 4) Sources Measurements   *")  
    print("*****************************")
    # 4) Source measurements
    # Source measurement
    sourceMeasurementTask.run(measCat=sources, exposure=calexp)

    # To get a better look at the output sources, 
    # we need to make sure that the `SourceCatalog` is contiguous in memory. 
    # Converting to an `astropy` table provides a human-readable output format. 
    # A deeper dive into `SourceCatalog` is beyond the scope of this notebook.
    #
    print("*************************************")
    print("* 5) Analyse the results in sources *")
    print("*************************************")
    
    sources = sources.copy(True)
    ts = sources.asAstropy()
    dfs = ts.to_pandas()
    x0 = x_target
    y0 = y_target
    distances = dfs[["slot_Centroid_x","slot_Centroid_y"]].apply(lambda row : np.sqrt( (row["slot_Centroid_x"]-x0)**2+(row["slot_Centroid_y"]-y0)**2),axis=1)
    row_sel = distances[distances == distances.min()]
    idxs_sel = row_sel.index[0]
    the_meassource_row = dfs.iloc[idxs_sel].dropna()
    print( the_meassource_row)
    all_sourcemeas.append(dict(the_meassource_row)) 
    
    if index>=30:
        break

In [ ]:
NIMG = len(all_cutout)
NCOLS = 3
#NROWS = int(np.ceil(NIMG/NCOLS))
NROWS = NIMG

In [ ]:
fig, axes = plt.subplots(ncols=NCOLS,nrows=NROWS,figsize=(6*NCOLS,6*NROWS))
for index,ax in enumerate(axes.flatten()):
    #if index//2 == NIMG:
    #    break

    # even number show the cutout
    if index%NCOLS  == 0:
        cutout = all_cutout[index//NCOLS].image.array
        visitId = listOfVisitId[index//NCOLS]
        #ax.imshow(cutout,interpolation="nearest")
        psfSigma = all_psfSigma[index//NCOLS]
        psfMagDiffMmag = all_psfMagDiffMmag[index//NCOLS]
        apFlux_50_0_instFlux = all_apFlux_50_0_instFlux[index//NCOLS]
        apFlux_50_0_instFluxErr = all_apFlux_50_0_instFluxErr[index//NCOLS]

        
        x_target,y_target = all_xytarget[index//NCOLS][0],all_xytarget[index//NCOLS][1]
        #textstr = '\n'.join((
        #r'$expos = %.0f$' % (visitId, ),
        #r'$x_t = %.0f , y_t = %.0f$' % (x_target,y_target),
        #r'$psfMagDiff = %.0f mmag$' %(psfMagDiffMmag, ),  
        #r'$\sigma_{PSF} = %.2f pix$' % (psfSigma , )))

        textstr = '\n'.join((
        r'$expos = %.0f$' % (visitId, ),
        r'$x_t = %.1f , y_t = %.1f$' % (x_target,y_target),
        r'$psfMagDiff = %.0f mmag$' %(psfMagDiffMmag, ),   
        r'$apFlux\_50\_instFlux = %.0f \;ADU$' %(apFlux_50_0_instFlux, ),  
        r'$apFlux\_50\_instFluxErr = %.0f \;ADU$' %(apFlux_50_0_instFluxErr, ),   
        r'$\sigma_{PSF} = %.2f pix$' % (psfSigma , )))
        
        ann = Annulus((boxSize,boxSize),r=psfSigma,width=0.1,color="red")
        ax.imshow(cutout,aspect='equal')
        ax.add_patch(ann)
        ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,
        verticalalignment='top',color="y", bbox=props)  
        title = f" {index//NCOLS}) {visitId}"
        ax.set_title(title)
    

        # 'base_PsfFlux_instFlux'
        # 'slot_ApFlux_instFlux'
        # 'base_GaussianFlux_instFlux'
        # 'base_CircularApertureFlux_50_0_instFlux'
        # 'base_PsfFlux_area'
        # 'base_PsfFlux_chi2'
        # 'base_PsfFlux_npixels'

        the_sourcemeas = all_sourcemeas[index//NCOLS]
        base_PsfFlux_instFlux = the_sourcemeas['base_PsfFlux_instFlux'] 
        slot_ApFlux_instFlux = the_sourcemeas['slot_ApFlux_instFlux']
        base_GaussianFlux_instFlux = the_sourcemeas['base_GaussianFlux_instFlux']
        base_CircularApertureFlux_50_0_instFlux = the_sourcemeas['base_CircularApertureFlux_50_0_instFlux']
        base_PsfFlux_area = the_sourcemeas['base_PsfFlux_area']
        base_PsfFlux_chi2 = the_sourcemeas['base_PsfFlux_chi2']
        base_PsfFlux_npixels = the_sourcemeas['base_PsfFlux_npixels']

        textstr1 = '\n'.join((
            r'$PsfFlux\_instFlux = %.0f$ ADU' % (base_PsfFlux_instFlux, ),
            r'$ApFlux\_instFlux = %.0f$ ADU' % (slot_ApFlux_instFlux, ),
            r'$GaussianFlux\_instFlux  = %.0f$ ADU' %(base_GaussianFlux_instFlux, ),  
            r'$CircApFlux50\_instFlux = %.0f$ ADU' % (base_CircularApertureFlux_50_0_instFlux , )))


        textstr2 = '\n'.join((
            r'$PsfFlux\_area = %.0f$' % (base_PsfFlux_area, ),
            r'$PsfFlux\_chi2 = %.0f$' % (base_PsfFlux_chi2, ),
            r'$PsfFlux\_npixels = %.0f$' % (base_PsfFlux_npixels, )))

    # odd number shows the heavy footprint  
    elif index%NCOLS  == 1:
        hfootpr = all_hfpspans[index//NCOLS]
        footpr = all_fpspans[index//NCOLS]
        ax.imshow(hfootpr,cmap="viridis",aspect='equal')
        title = all_hfp[index//NCOLS]
        ax.set_title(title)
        ax.text(0.05, 0.95, textstr1, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="y", bbox=props) 
    elif index%NCOLS == 2:
        mask = all_hfpspans_mask[index//NCOLS]
        im = ax.imshow(mask,cmap="viridis",vmin=0,vmax=32+16,aspect='equal')
        #ax = plt.gca()
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad="1%")
        ax.set_title("pixel mask")
        plt.colorbar(im, cax=cax, ticks=[0, 32, 32+16])
        ax.text(0.2, 0.55, textstr2, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="r", bbox=props) 
        


plt.suptitle(suptitle,y=1.0)
plt.tight_layout()
plt.show()

The values are the exponent of the bitmask. So pixels only marked detected will be 2^5 = 32. Pixels that are both on the edge of the original image and detected will be 2^5 + 2^4 = 48. We will visualize the mask plane values in a similar manner as before, except that we will be displaying the values of the mask array.

In [ ]:
calexp.getMask().getMaskPlaneDict()

In [ ]:
#mask = calexp.getMask()
#for maskName, maskBit in mask.getMaskPlaneDict().items():
#    print('{}: {}'.format(maskName, display.getMaskPlaneColor(maskName)))

In [ ]:
# The copy makes sure that the sources are sequential in memory
sources = sources.copy(True)

# Investigate the output source catalog
t = sources.asAstropy()
dfs = t.to_pandas()
t

In [ ]:
sources.getSchema().getNames(topOnly=True)

In [ ]:
sources.schema.getNames()

In [ ]:
sources.getSchema().find('id')

In [ ]:
sources.getSchema().checkUnits()

In [ ]:
dir(sources.getSchema())

In [ ]:
sources.getSchema().asAstropy()

In [ ]:
dfs

In [ ]:
x0 = x_target
y0 = y_target

In [ ]:
target0 = lsst.geom.Point2D(x0,y0)

In [ ]:
t[["slot_Centroid_x","slot_Centroid_y"]]

In [ ]:
distances = dfs[["slot_Centroid_x","slot_Centroid_y"]].apply(lambda row : np.sqrt( (row["slot_Centroid_x"]-x0)**2+(row["slot_Centroid_y"]-y0)**2),axis=1)

In [ ]:
row_sel = distances[distances == distances.min()]
row_sel

In [ ]:
row_sel.index[0]

In [ ]:
row_sel = dfs.iloc[17].dropna()
print(row_sel)

In [ ]:
dict(row_sel)

In [ ]:
# 'base_PsfFlux_instFlux'
# 'slot_ApFlux_instFlux'
# 'base_GaussianFlux_instFlux'
# 'base_CircularApertureFlux_50_0_instFlux'
# 'base_PsfFlux_area'
# 'base_PsfFlux_chi2'
# 'base_PsfFlux_npixels'